<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Handling dask arrays

`apply_ufunc` is a more advanced wrapper that is designed to apply functions
that expect and return NumPy (or other arrays). For example, this would include
all of SciPy's API. Since `apply_ufunc` operates on lower-level NumPy or Dask
objects, it skips the overhead of using Xarray objects making it a good choice
for performance-critical functions.

`apply_ufunc` can be a little tricky to get right since it operates at a lower
level than `map_blocks`. On the other hand, Xarray uses `apply_ufunc` internally
to implement much of its API, meaning that it is quite powerful!

Learning goals:
- Learn that `apply_ufunc` automates aspects of applying computation functions that are designed for pure arrays (like numpy arrays) on xarray objects

## Setup

In [ ]:
import dask
import numpy as np
import xarray as xr

First lets set up a `LocalCluster` using [dask.distributed](https://distributed.dask.org/).

You can use any kind of dask cluster. This step is completely independent of
xarray. While not strictly necessary, the dashboard provides a nice learning
tool.


In [ ]:
from dask.distributed import Client

client = Client()
client

<p>&#128070</p> Click the Dashboard link above. Or click the "Search" button in the dashboard.

Let's test that the dashboard is working..


In [ ]:
import dask.array

dask.array.ones((1000, 4), chunks=(2, 1)).compute()  # should see activity in dashboard

Let's open a dataset. We specify `chunks` so that we create a dask arrays for the DataArrays

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature", chunks={"time": 100})
ds

## A simple example

All the concepts from applying numpy functions carry over.

However the handling of dask arrays needs to be explicitly activated.

In [ ]:
# Expect an error here
def squared_error(x, y):
    return (x - y) ** 2


xr.apply_ufunc(squared_error, ds.air, 1)

There are two options for the `dask` kwarg.

1. `dask="allowed"` Dask arrays are passed to the user function. This is a good
   choice if your function can handle dask arrays and won't call compute
   explicitly.
2. `dask="parallelized"`. This applies the user function over blocks of the dask
   array using `dask.array.blockwise`. This is useful when your function cannot
   handle dask arrays natively (e.g. scipy API).

Since `squared_error` can handle dask arrays without computing them, we specify
`dask="allowed"`.


In [ ]:
sqer = xr.apply_ufunc(
    squared_error,
    ds.air,
    1,
    dask="allowed",
)
sqer  # dask-backed DataArray! with nice metadata!

Let's again use the wrapper trick to understand what `squared_error` receives.

We see that it receives a dask array.

In [ ]:
def wrapper(x, y):
    print(f"received x of type {type(x)}, shape {x.shape}")
    print(f"received y of type {type(y)}")
    return squared_error(x, y)


xr.apply_ufunc(wrapper, ds.air, 1, dask="allowed")

## Reductions and core dimensions

`squared_error` operated on a per-element basis. How about a reduction like `np.mean`?

Such functions involve the concept of "core dimensions". One way to think about core dimensions is to consider the smallest dimensionality of data necessary to apply the function.

For using more complex operations that consider some array values collectively,
it’s important to understand the idea of **core dimensions**. 
Usually, they correspond to the fundamental dimensions over
which an operation is defined, e.g., the summed axis in `np.sum`. A good clue
that core dimensions are needed is the presence of an `axis` argument on the
corresponding NumPy function.

With `apply_ufunc`, core dimensions are recognized by name, and then moved to
the last dimension of any input arguments before applying the given function.
This means that for functions that accept an `axis` argument, you usually need
to set `axis=-1`

Let's use `dask.array.mean` as an example of a function that can handle dask
arrays and uses an `axis` kwarg


In [ ]:
def time_mean(da):
    return xr.apply_ufunc(
        dask.array.mean,
        da,
        input_core_dims=[["time"]],
        dask="allowed",
        kwargs={"axis": -1},  # core dimensions are moved to the end
    )


time_mean(ds.air)

Again, this identical to the built-in `mean`

In [ ]:
ds.air.mean("time").identical(time_mean(ds.air))

## Automatically parallelizing dask-unaware functions

A very useful `apply_ufunc` feature is the ability to apply arbitrary functions
in parallel to each block. This ability can be activated using
`dask="parallelized"`. Again xarray needs a lot of extra metadata, so depending
on the function, extra arguments such as `output_dtypes` and `output_sizes` may
be necessary.

We will use `scipy.integrate.trapz` as an example of a function that cannot
handle dask arrays and requires a core dimension. If we call `trapz` with a dask
array, we get a numpy array back that is, the values have been eagerly computed.
This is undesirable behaviour


In [ ]:
import scipy as sp
import scipy.integrate

sp.integrate.trapz(ds.air.data, axis=ds.air.get_axis_num("lon"))  # does NOT return a dask array

In [ ]:
xr.apply_ufunc(
    sp.integrate.trapz,
    ds,
    input_core_dims=[["lon"]],
    kwargs={"axis": -1},
    dask="parallelized",
)

In [ ]:
client.close()

## More

1. https://xarray.pydata.org/en/stable/examples/apply_ufunc_vectorize_1d.html#
2. https://docs.dask.org/en/latest/array-best-practices.html
